# Random Forest: Predicting CHD

Here is an intersting problem of understanding what factors contribute to CHD and can CHD be predicted by building an analytical model.

The next two sections will introduce some basics of CHD, where the dataset is derived from and what are the attributes available in the dataset.

### What is coronary heart disease?


[Coronary heart disease (CHD)](https://en.wikipedia.org/wiki/Coronary_artery_disease)  is when your coronary arteries (the arteries that supply your heart muscle with oxygen-rich blood) become narrowed by a gradual build-up of fatty material within their walls. These arteries can become narrowed through build-up of plaque, which is made up of cholesterol and other substances. Narrowed arteries can cause symptoms, such as chest pain (angina), shortness of breath, and fatigue.


### Dataset Description

Data is avaialable at: http://statweb.stanford.edu/~tibs/ElemStatLearn/datasets/
And header informtion is available at: http://statweb.stanford.edu/~tibs/ElemStatLearn/datasets/SAheart.info.txt

A retrospective sample of **males in a heart-disease high-risk region of the Western Cape, South Africa**. There are roughly two controls per case of CHD. Many of the CHD positive men have undergone blood pressure reduction treatment and other programs to reduce their risk factors after their CHD event. In some cases the measurements were made after these treatments. These data are taken from a larger dataset, described in Rousseauw et al, 1983, South African Medical Journal. 

### Import and load the dataset

In [ ]:
import pandas as pd
import numpy as np

np.random.seed(100)

In [ ]:
saheart_ds = pd.read_csv( "https://drive.google.com/uc?export=download&id=1yRyZMfBQ8anG10GDFsLf15GRYPQqN12b" )

In [ ]:
saheart_ds.head()

In [ ]:
saheart_ds.columns

In [ ]:
saheart_ds.info()

In [ ]:
saheart_ds.chd.value_counts(normalize=True)

The class label int the column **chd** indicates if the person has a coronary heart disease: negative (0) or positive (1). 

Attributes description: 
- **sbp**:          systolic blood pressure 
- **tobacco**:      cumulative tobacco (kg) 
- **ldl**:          low densiity lipoprotein cholesterol 
- **adiposity**:    the size of the hips compared to the person's height 
- **famhist**:      family history of heart disease (Present, Absent) 
- **typea**:        type-A behavior 
- **obesity**:      BMI index
- **alcohol**:      current alcohol consumption 
- **age**:          age at onset

### Encoding Categorical Features

In [ ]:
saheart_model_df = pd.get_dummies( saheart_ds, 
                                   columns = ['famhist'],
                                   drop_first = True)

In [ ]:
saheart_model_df.head( 10 )

In [ ]:
saheart_model_df = saheart_model_df.drop( "row.names", axis = 1 )

In [ ]:
saheart_model_df.columns

## Splitting Dataset into Train and Test

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
feature_cols = ['sbp', 'tobacco', 'ldl', 
                'adiposity', 'typea', 
                'obesity', 'alcohol', 
                'age', 'famhist_Present' ]

train_X, test_X, train_y, test_y = train_test_split(saheart_model_df[feature_cols],
                                                    saheart_model_df['chd'],
                                                    test_size = 0.3,
                                                    random_state = 42 ) 

In [ ]:
len( train_X )

In [ ]:
len( test_X )

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
class_weights = compute_class_weight(class_weight = "balanced", 
                                     classes = np.unique(train_y), 
                                     y = train_y)
class_weights

In [ ]:
class_weight={1:1.45, 0:0.761}

## Building Logistic Regression Model

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
log_clf = LogisticRegression(random_state = 100,
                             class_weight=class_weight,
                             max_iter=1000)

In [ ]:
log_clf.fit(train_X, train_y)

### Predicting in test set using the model

In [ ]:
logreg_test_results = pd.DataFrame( { 'actual':  test_y, 
                                      'predicted': log_clf.predict( test_X ) } )

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(logreg_test_results.actual,
                            logreg_test_results.predicted))

## Building SVM Model

In [ ]:
from sklearn.svm import SVC

rbf_svc = SVC(kernel = 'rbf', class_weight=class_weight, probability=True)

In [ ]:
rbf_svc.fit(train_X, train_y)

In [ ]:
svc_test_results = pd.DataFrame( {'actual':  test_y, 
                                  'predicted': rbf_svc.predict( test_X ) } )

In [ ]:
print(classification_report(svc_test_results.actual,
                            svc_test_results.predicted))

## Building Decision Tree Model

In [ ]:
from sklearn.tree import DecisionTreeClassifier

tree_clf = DecisionTreeClassifier(max_depth = 7, 
                                  class_weight=class_weight, 
                                  criterion='gini')

tree_clf.fit(train_X, train_y)

In [ ]:
tree_test_results = pd.DataFrame( { 'actual':  test_y, 
                                    'predicted': tree_clf.predict( test_X ) } )
                                       
                                       
print(classification_report(tree_test_results.actual,
                            tree_test_results.predicted))

## Building Voting Classifier Model

In [ ]:
from sklearn.ensemble import VotingClassifier

In [ ]:
clf_list = [('log', log_clf), 
            ('rf', rf_clf), 
            ('tree', tree_clf)]

In [ ]:
vote_clf = VotingClassifier(estimators = clf_list, 
                            voting='soft')

In [ ]:
vote_clf.fit(train_X, train_y)

In [ ]:
voting_test_results = pd.DataFrame( { 'actual':  test_y, 
                                      'predicted': vote_clf.predict( test_X ) } )

In [ ]:
print(classification_report(voting_test_results.actual,
                            voting_test_results.predicted))